In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN")
login(token = hf_token)

In [2]:
base_model = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"
new_model = "/kaggle/input/llama-3-fine-tune-math-problems/llama-3.1-8b-chat-math-teacher/"

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format

## Note
Depending on the base model size, the following commands may require a GPU with more memory or multiple GPUs

In [ ]:
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        offload_buffers=True,
)

base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)

model = model.merge_and_unload()

In [7]:
messages = [{"role": "user", "content": "Find the sum of all multiples of 9 that are less than 100."}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|im_start|>user
Find the sum of all multiples of 9 that are less than 100.<|im_end|>
<|im_start|>assistant
To find the sum of all multiples of 9 that are less than 100, we first need to identify the last multiple of 9 that is less than 100. Since 99 is the largest multiple of 9 less than 100, we will use that number.

The formula to find the sum of an arithmetic series is:

Sum = n/2 * (first term + last term)

Where n is the number of terms in the series.

The first term is the first multiple of 9, which is 9, and the last term is 99. To find the number of terms (n), we can use the formula:

n = (last term - first term) / common difference + 1

In this case, the common difference is 9, since we are adding 9 to get from one multiple to the next.

So, let's calculate:

n = (99 - 9) / 9 + 1
n = 90 / 9 + 1
n = 10 + 1
n = 11

Now we can find the sum:

Sum = 11/2 * (9 + 99)
Sum = 11/2 * 108
Sum = 5.5 * 108
Sum = 594

Therefore, the sum of all multiples of 9 that are less than 100 is 594.
a

In [8]:
model.save_pretrained("llama-3.1-8b-chat-math-teacher")
tokenizer.save_pretrained("llama-3.1-8b-chat-math-teacher")

('llama-3.1-8b-chat-math-teacher/tokenizer_config.json',
 'llama-3.1-8b-chat-math-teacher/special_tokens_map.json',
 'llama-3.1-8b-chat-math-teacher/tokenizer.json')

In [ ]:
model.push_to_hub("llama-3.1-8b-chat-math-teacher", use_temp_dir=False)
tokenizer.push_to_hub("llama-3.1-8b-chat-math-teacher", use_temp_dir=False)